In [8]:
import pandas as pandas
import numpy as np

In [9]:
# load 'test.xlsx'

df_test = pandas.read_excel('test.xlsx', sheet_name='data', header=None)

df_test

,0,1,2
0,Community,Community Name,Braybrook (Suburb)
1,Community,Region,Northern and Western Metropolitan
2,Geography,Map reference,4
3,Geography,Grid reference,A3
4,Geography,Location,10km WNW of Melbourne
...,...,...,...
221,Hospital,Distance to nearest public hospital with emerg...,10.161988
222,Hospital,Presentations to emergency departments due to ...,543.631989
223,Hospital,Presentations to emergency departments due to ...,20.647263
224,Hospital,Category 4 & 5 emergency department presentations,1683.966712


In [10]:
# set the first column as reference
ref_col = df_test.iloc[:, 0]

In [11]:
def excel_loader(xlsx_path, sheet_name='data', ref_col=ref_col):
    # load the excel file
    df = pandas.read_excel(xlsx_path, sheet_name=sheet_name, header=None)
    # replace the first column with the reference column
    df.iloc[:, 0] = ref_col
    # change the column names
    df.columns = ['feature_kind', 'feature_name', 'feature_value']
    return df



In [12]:
excel_loader('data/Ascot-Vale-Suburb - XLSX.xlsx')

,feature_kind,feature_name,feature_value
0,Community,Community Name,Ascot Vale (Suburb)
1,Community,Region,Northern and Western Metropolitan
2,Geography,Map reference,4
3,Geography,Grid reference,B3
4,Geography,Location,6km NW of Melbourne
...,...,...,...
221,Hospital,Distance to nearest public hospital with emerg...,4.993841
222,Hospital,Presentations to emergency departments due to ...,679.257076
223,Hospital,Presentations to emergency departments due to ...,20.502456
224,Hospital,Category 4 & 5 emergency department presentations,1864.918123
